## FS-PDDLVM example

\begin{align}
-\frac{\partial}{\partial x}\left(k(u,x)\frac{\partial u(x)}{\partial x}\right) &= w(x) \quad \forall x \in [-1,1]\\
u(-1) &= u(1) = 0 \\
k(u,x) &:= \log\left(1 + \exp\left(u(x)\sum_{i}z_{i}\phi_{i}(x)\right)\right) + 0.1 \\
\end{align}

In [ ]:
import os
import tensorflow as tf
import tensorflow_probability as tfp
tfd = tfp.distributions
os.environ['CUDA_VISIBLE_DEVICES']='-1'

import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

import fenics as fe
import numpy as np
import matplotlib.pyplot as plt
import time
import gc
plt.rcParams["figure.figsize"] = (10,8)

In [ ]:
dirname='data-groundTruth_2--PoissonNonLinear1D_1000'
os.makedirs(dirname, exist_ok=True)


In [ ]:
def solve_nonlinear_poisson(diffusion_data, w, nx=101, degree=1, solver_options={"conv_criterion": "incremental", "rtol": 1e-6, "report": True, "max_it": 50}):
    
    # create domain, FEM space, and boundary conditions
    mesh = fe.IntervalMesh(nx,-1.0,1.0)
    mesh = fe.IntervalMesh(nx,-1.0,1.0)
    V = fe.FunctionSpace(mesh, "CG", degree)
    xgrid = V.tabulate_dof_coordinates()[:,0]
    bc = fe.DirichletBC(V, fe.Constant(0), "on_boundary")
    
    # create diffusion function
    def k(u,x):
        my_exponent = 0.0
        for (z,phi) in diffusion_data:
            my_exponent += z * phi(x[0])
        my_exponent = u * my_exponent
        return fe.function.function.ufl.ln(1.0 + fe.function.function.ufl.exp(my_exponent)) + 0.1
    
    # set up non-linear variational problem
    x = fe.function.function.ufl.SpatialCoordinate(mesh)
    uh = fe.Function(V)
    v = fe.TestFunction(V)
    if w == 0.0:
        F = k(uh,x)*fe.dot(fe.grad(uh), fe.grad(v))*fe.dx
    else:
        F = k(uh,x)*fe.dot(fe.grad(uh), fe.grad(v))*fe.dx - w*v*fe.dx
    
    J = fe.derivative(F,uh)

    fe.solve(F == 0, uh, J=J, bcs=bc)

    xSln = np.array( xgrid )
    uSln = np.array( uh.vector().get_local() )
    
    del mesh
    del V
    del bc
    del x, uh, v, F, J

    return uSln, xSln

## Very simple example

In [ ]:
diffusion_data = [(1.0,lambda x: x**0)]
w = 1.0
uh, xgrid = solve_nonlinear_poisson(diffusion_data, w)

plt.plot(xgrid, uh)
plt.show()
print('xgrid.shape', xgrid.shape)

## Chebyshev expansion test example

\begin{align}
z_0 &= 1 \\
z_1 &= -1 \\
z_2 &= 0.1 \\
\phi_{i}(x) &:= T_{i}(x) = \cos(i * \arccos(x))
\end{align}

In [ ]:
def cheby_T(x,n):
    return fe.function.function.ufl.cos(n * fe.function.function.ufl.acos(x))

In [ ]:
diffusion_data = [(1.0, lambda x: cheby_T(x,0)), (-1.0, lambda x: cheby_T(x,1)), (0.1, lambda x: cheby_T(x,2))]
w = 5.0

uh, xgrid = solve_nonlinear_poisson(diffusion_data, w, nx=200)

plt.plot(xgrid, uh)
plt.show()
print('xgrid.shape', xgrid.shape)
np.savetxt(dirname+'/' + 'FenicsMeshX.dat'       , xgrid )

In [ ]:
dimZ = 5
dimW = 1

zDist = tfd.Uniform(low=tf.constant(-1., shape=[dimZ]), high=tf.constant(1., shape=[dimZ]))
wDist = tfd.Uniform(low=tf.constant( 1., shape=[dimW]), high=tf.constant(2., shape=[dimW]))

nSamples  = 1_000
allTrueZs = zDist.sample(nSamples)
allTrueWs = wDist.sample(nSamples)

groundTruthU = []
groundTruthX = xgrid

startTime = time.time()
for i in range(allTrueZs.shape[0]):
        
    
    diffusion_data=  [ (allTrueZs[i][0].numpy().squeeze(), lambda x: cheby_T(x,0)), 
                       (allTrueZs[i][1].numpy().squeeze(), lambda x: cheby_T(x,1)),  
                       (allTrueZs[i][2].numpy().squeeze(), lambda x: cheby_T(x,2)),
                       (allTrueZs[i][3].numpy().squeeze(), lambda x: cheby_T(x,3)),
                       (allTrueZs[i][4].numpy().squeeze(), lambda x: cheby_T(x,4))
                     ]
        
    uSln, xSln = solve_nonlinear_poisson(diffusion_data, allTrueWs[i].numpy().squeeze(), nx=200)
    
    gc.collect()
    
    plt.plot(xSln, uSln)
    
    groundTruthU.append(np.copy(uSln) )

print('time taken = ', time.time() - startTime, 's')
plt.show()

np.savetxt(dirname+'/' + 'allGroundTruthU.dat'   , groundTruthU )
np.savetxt(dirname+'/' + 'allGroundTruthZ.dat'   , allTrueZs )
np.savetxt(dirname+'/' + 'allGroundTruthW.dat'   , allTrueWs )
np.savetxt(dirname+'/' + 'FenicsMeshX.dat'       , xSln )